## 09) 순환 신경망(Recurrent Neural Network)
* 피드 포워드 신경망은 입력의 길이가 고정되어 있어 한계 있음
* 다양한 길이의 입력 시퀀스를 처리할 수 있는 인공 신경망 필요  


### 1) RNN
* RNN은 시퀀스 Sequence 모델. 입력과 출력을 시퀀스 단위로 처리하는 모델.
* 재귀 신경망(Recursive Neural Network)는 전혀 다름

#### 1. RNN
* 피드 포워드 신경망: 전부 은닉층에서 활성화 함수를 지난 값은 오직 출력층 방향만 향함
* BUT, RNN은 은닉층의 노드에서 활성화 함수를 통해 나온 결과값을 출력층 방향으로도 보내면서, **다시 은닉층 노드의 다음 계산의 입력으로 보내는 특징 가짐**  


* 메모리 셀, RNN 셀: 은닉층에서 활성화 함수를 통해 결과를 내보내는 역할을 하는 노드. 이전의 값을 기억하려고 하는 일종의 메모리 역할.
* 은닉층의 메모리 셀: 각각의 시점 time step에서 바로 이전 시점에서의 은닉층 메모리 셀에서 나온 값을 자신의 입력으로 하는 재귀적 활동
* **은닉 상태 hidden state**: 메모리 셀이 출력층 방향으로 또는 다음 시점 t+1의 자신에게 보내는 값
* 피드 포워드 신경망: 뉴런, RNN: 뉴런보다는 입력 벡터, 출력 벡터, 은닉 상태  


* 입출력 길이 다르게 가능
  * 가장 보편적 입출력 단위는 단어 벡터
  * ex) one-to-many: 이미지 캡셔닝(하나의 이미지 입력에 대해서 사진의 제목 출력)
  * ex) many-to-one: 감성 분류, 스팸 메일 분류
  * ex) many-to-many: 입력 문장으로부터 대답 문장 출력하는 챗봇. 입력 문장으로부터 번역된 문장을 출력하는 번역기. 개체명 인식. 품사 태깅.
* ht를 계산하기 위한 활성화 함수: tanh, ReLU

#### 2. Keras로 RNN 구현


In [ ]:
# RNN층을 추가하는 코드
model.add(SimpleRNN(hidden_size))

* hidden_size: 은닉 상태의 크기 정의. output_dim과 동일. RNN의 용량을 늘린다.. 128, 256, 512, 1024
* output_dim: 메모리 셀이 다음 시점의 메모리 셀과 출력층으로 보내는 값의 크기
* timesteps: 입력 시퀀스의 길이(input_length). 시점의 수.
* input_dim: 입력의 크기

In [ ]:
# 추가 인자 사용
model.add(SimpleRNN(hidden_size, input_shape=(timesteps, input_dim)))

# 다른 표기
model.add(SimpleRNN(hidden_size, input_length=M, input_dim=N))

* RNN 층은 (batch_size, timesteps, input_dim) 크기의 3D 텐서를 입력으로
* 위 코드의 결과는 출력층이 아니라 하나의 은닉 상태  

* if 은닉 상태만 리턴, (batch_size, output_dim) 크기의 2D 텐서
* if 전체 시퀀스 리턴, (batch_size, timesteps, output_dim) 크기의 3D 텐서를 리턴
  * 이는 RNN 층의 return_sequences 매개 변수에 True를 설정하여 설정 가능  

* if 마지막 은닉 상태만 전달, many-to-one 문제 해결
* if 모든 시점의 은닉 상태 전달, 다음 층에 은닉층이 하나 더 있거나 many-to-many 해결 가능



In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN

model = Sequential()
model.add(SimpleRNN(3, input_shape=(2,10))) #3은 은닉 상태 크기
# model.add(SimpleRNN(3, input_length=2, input_dim=10))와 동일함.
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 3)                 42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


In [2]:
# batch size 정의
model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8,2,10)))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (8, 3)                    42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


In [3]:
# return_sequences 매개 변수에 True를 기재
# 출력값으로 (batch_size, timesteps, output_dim) 크기의 3D 텐서를 리턴

model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8,2,10), return_sequences=True))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (8, 2, 3)                 42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


#### 3. 파이썬으로 RNN 구현
* 직접 Numpy로 RNN 층 구현

In [ ]:
# 아래의 코드는 의사 코드(pseudocode)로 실제 동작하는 코드가 아님

hidden_state_t = 0 # 초기 은닉 상태를 0(벡터)로 초기화
for input_t in input_length: # 각 시점마다 입력을 받음
    output_t = tanh(input_t, hidden_state_t) # 각 시점에 대해서 입력과 은닉 상태를 가지고 연산
    hidden_state_t = output_t # 계산 결과는 현재 시점의 은닉 상태

In [4]:
import numpy as np

timesteps = 10   # 시점의 수. 문장의 길이.
input_dim = 4   # 입력의 차원. 단어 벡터의 차원.
hidden_size = 8   # 은닉 상태의 크기. 메모리 셀의 용량.

# 입력에 해당되는 2D 텐서
inputs = np.random.random((timesteps, input_dim))

# 초기 은닉 상태는 0(벡터)로 초기화
hidden_state_t = np.zeros((hidden_size,)) 

In [5]:
print(hidden_state_t)

[0. 0. 0. 0. 0. 0. 0. 0.]


* **Wx: 은닉 상태의 크기 x 입력의 차원**
* ** Wh: 은닉 상태의 크기 x 은닉 상태의 크기**
* ** b: 은닉 상태의 크기**

In [6]:
# 가중치와 편향

Wx = np.random.random((hidden_size, input_dim))  # (8, 4)크기의 2D 텐서 생성. 입력에 대한 가중치.
Wh = np.random.random((hidden_size, hidden_size)) # (8, 8)크기의 2D 텐서 생성. 은닉 상태에 대한 가중치.
b = np.random.random((hidden_size,)) # (8,)크기의 1D 텐서 생성. 이 값은 편향(bias).

In [7]:
print(np.shape(Wx))
print(np.shape(Wh))
print(np.shape(b))

(8, 4)
(8, 8)
(8,)


In [8]:
total_hidden_states = []

# 메모리 셀 동작
for input_t in inputs: # 각 시점에 따라서 입력값이 입력됨.
  output_t = np.tanh(np.dot(Wx,input_t) + np.dot(Wh,hidden_state_t) + b) # Wx * Xt + Wh * Ht-1 + b(bias)
  total_hidden_states.append(list(output_t)) # 각 시점의 은닉 상태의 값을 계속해서 축적
  print(np.shape(total_hidden_states)) # 각 시점 t별 메모리 셀의 출력의 크기는 (timestep, output_dim)
  hidden_state_t = output_t

# 출력 시 값을 깔끔하게 해주는 용도.
total_hidden_states = np.stack(total_hidden_states, axis = 0) 

# (timesteps, output_dim)
print(total_hidden_states)

(1, 8)
(2, 8)
(3, 8)
(4, 8)
(5, 8)
(6, 8)
(7, 8)
(8, 8)
(9, 8)
(10, 8)
[[0.83400137 0.93416315 0.95592632 0.74883609 0.75823877 0.90855195
  0.96356391 0.98224485]
 [0.99950465 0.99984241 0.99998116 0.99912536 0.99995231 0.99891907
  0.99998284 0.99997251]
 [0.99991506 0.99997479 0.99999777 0.99991291 0.99998981 0.99944575
  0.99999753 0.99999711]
 [0.99974104 0.99992473 0.99999279 0.99978429 0.99998499 0.99908333
  0.9999891  0.99998378]
 [0.99989641 0.99993846 0.99999275 0.99950951 0.99997006 0.99943216
  0.99999598 0.99999356]
 [0.99978581 0.99987362 0.9999771  0.99945846 0.99995102 0.9987945
  0.9999861  0.99997351]
 [0.99986078 0.9999563  0.99999427 0.99979328 0.99998206 0.99922245
  0.99999638 0.99999173]
 [0.99993001 0.99996484 0.99999765 0.99982538 0.99998651 0.99960933
  0.99999686 0.99999779]
 [0.99984847 0.99991956 0.99999109 0.99962776 0.99997296 0.99925958
  0.99999168 0.99998955]
 [0.99985708 0.99994578 0.99999322 0.99968647 0.99997846 0.99928214
  0.99999581 0.999991  ]]

#### 4. Deep RNN

In [ ]:
# 은닉층 2개 추가
model = Sequential()
model.add(SimpleRNN(hidden_size, input_length=10, input_dim=5, return_sequences=True))
model.add(SimpleRNN(hidden_size, return_sequences=True))

#### 5. 양방향 RNN
* 시점 t에서의 출력값을 예측할 때 이전 시점의 데이터뿐만 아니라, 이후 데이터로도 예측할 수 있다는 아이디어에 기반
* 하나의 출력값을 예측하기 위해 기본적으로 두 개의 메모리 셀을 사용
  * **앞 시점의 은닉 상태(Forward States), 뒤 시점의 은닉 상태(Backward States)**

In [ ]:
from tensorflow.keras.models import Bidirectional

timesteps = 10
input_dim = 5

model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences = True), input_shape=(timesteps, input_dim)))

### 2) 장단기 메모리 Long Short-Term Memory, LSTM
* ** 장기 의존성 문제 The problem of Long-Term Dependencies ** 
  * 바닐라 RNN의 단점으로 비교적 짧은 sequence에 대해서만 효과 보임. 시점 time step이 길어질 수록 앞의 정보가 뒤로 충분히 전달되지 못함.  


* LSTM은 은닉층의 메모리 셀에 **입력 게이트, 망각 게이트, 출력 게이트**를 추가하여 불필요한 기억을 지우고, 기억해야 할 것들을 정함.
  * **은닉 상태 hidden state**를 계산하는 식이 전통 RNN보다 복잡
  * **셀 상태 cell state**라는 값을 추가

1. 입력 게이트: 현재 정보를 기억하기 위한 게이트
2. 삭제 게이트: 기억을 삭제하기 위한 게이트
3. 셀 상태(장기 상태)
4. 출력 게이트와 은닉 상태(단기 상태)

구체적인 설명은 https://wikidocs.net/22888 를 참고하면 됨


### 3) 게이트 순환 유닛 Gated Recurrent Unit, GRU
* 은닉 상태를 업데이트 하는 계산을 줄임
* LSTM에는 출력, 입력, 삭제 게이트가 존재하지만, GRU에서는 **업데이트 게이트와 리셋 게이트** 두 가지 게이트만 존재
* 데이터 양이 적을 때는 매개 변수의 양이 적은 GRU
* 데이터 양이 더 많으면 LSTM이 더 나음

In [ ]:
model.add(GRU(hidden_size, input_shape=(timesteps, input_dim)))

### 4) Keras의 SimpleRNN과 LSTM 이해하기
#### 1. 임의의 입력 생성하기

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN, LSTM, Bidirectional

# 단어 벡터 차원 5, 문장의 길이 4
train_X = [[0.1, 4.2, 1.5, 1.1, 2.8], [1.0, 3.1, 2.5, 0.7, 1.1], [0.3, 2.1, 1.5, 2.1, 0.1], [2.2, 1.4, 0.5, 0.9, 1.1]]
print(np.shape(train_X))

(4, 5)


* 4번의 시점 timesteps이 존재하고, 각 시점마다 5차원의 단어 벡터가 입력으로 사용됨

In [2]:
#  RNN은 2D 텐서가 아니라 3D 텐서를 입력을 받음
# 위에서 만든 2D 텐서를 3D 텐서로 변경합니다. 이는 배치 크기 1을 추가하여 해결

train_X = [[[0.1, 4.2, 1.5, 1.1, 2.8], [1.0, 3.1, 2.5, 0.7, 1.1], [0.3, 2.1, 1.5, 2.1, 0.1], [2.2, 1.4, 0.5, 0.9, 1.1]]]
train_X = np.array(train_X, dtype=np.float32)
print(train_X.shape)

(1, 4, 5)


####  2. SimpleRNN 이해하기
* 대표적인 인자로 return_sequences와 return_state가 있음

In [3]:
rnn = SimpleRNN(3)
# rnn = SimpleRNN(3, return_sequences=False, return_state=False)와 동일.
hidden_state = rnn(train_X)

print('hidden state : {}, shape: {}'.format(hidden_state, hidden_state.shape))

hidden state : [[-0.61090666 -0.7983958  -0.61283493]], shape: (1, 3)


* (1,3)은 마지막 시점의 은닉 상태. 은닉 상태의 크기를 3으로 지정함.
* return_sequenes가 False면 마지막 시점의 은닉 상태만 출력
* True면 모든 시점의 은닉 상태만 출력

In [4]:
rnn = SimpleRNN(3, return_sequences=True)
hidden_states = rnn(train_X)

print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))

hidden states : [[[ 0.9649639   0.78921616 -0.9214914 ]
  [ 0.92051166  0.90270686 -0.90345895]
  [ 0.89259624  0.14309247 -0.8523793 ]
  [-0.06274243  0.38294312 -0.9980742 ]]], shape: (1, 4, 3)


* (1, 4, 3) 크기의 텐서가 출력됨
* 입력 데이터는 (1, 4, 5)의 크기를 가지는 3D 텐서. 
  * 4가 시점 timesteps에 해당하는 값이므로 모든 시점에 대해서 은닉 상태의 값을 출력하여 (1, 4, 3) 크기의 텐서를 출력하는 것
* return_state가 True면 return_sequences의 여부와 상관 없이 *마지막 시점의 은닉 상태 출력*
* return_sequences = True, return_state = True ==> 두 개의 출력 리턴

In [5]:
rnn = SimpleRNN(3, return_sequences=True, return_state=True)
hidden_states, last_state = rnn(train_X)

print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('last hidden state : {}, shape: {}'.format(last_state, last_state.shape))

hidden states : [[[-0.9633086   0.999996    0.21033907]
  [ 0.5332108   0.9999877  -0.83054066]
  [-0.8734952   0.99850875 -0.34791538]
  [ 0.47813833  0.99915427  0.52087396]]], shape: (1, 4, 3)
last hidden state : [[0.47813833 0.99915427 0.52087396]], shape: (1, 3)


* 첫번째는 return_sequences = True로 인한 모든 시점의 은닉 상태
* 두번째는 return_state = True로 인한 마지막 시점의 은닉 상태

In [6]:
# return_sequences = False, return_state = True인 경우?
rnn = SimpleRNN(3, return_sequences=False, return_state=True)
hidden_state, last_state = rnn(train_X)

print('hidden state : {}, shape: {}'.format(hidden_state, hidden_state.shape))
print('last hidden state : {}, shape: {}'.format(last_state, last_state.shape))

hidden state : [[-0.770695   -0.990965    0.65312725]], shape: (1, 3)
last hidden state : [[-0.770695   -0.990965    0.65312725]], shape: (1, 3)


* 두 개의 출력 모두 마지막 시점의 은닉 상태 출력

### 3. LSTM 이해하기
* return_sequences = False, return_state = True

In [7]:
lstm = LSTM(3, return_sequences=False, return_state=True)
hidden_state, last_state, last_cell_state = lstm(train_X)

print('hidden state : {}, shape: {}'.format(hidden_state, hidden_state.shape))
print('last hidden state : {}, shape: {}'.format(last_state, last_state.shape))
print('last cell state : {}, shape: {}'.format(last_cell_state, last_cell_state.shape))

hidden state : [[ 0.057607    0.07817724 -0.24318294]], shape: (1, 3)
last hidden state : [[ 0.057607    0.07817724 -0.24318294]], shape: (1, 3)
last cell state : [[ 0.13276935  0.6909851  -1.2508996 ]], shape: (1, 3)


* SimpleRNN과 달리, 세 개의 결과 반환
* 첫 번째 결과: return_sequences = False이므로 마지막 시점의 은닉 상태
* **그런데 LSTM이 SimpleRNN과 다른 점은 return_state = True인 경우에는 마지막 시점의 은닉 상태 뿐만 아니라 셀 상태도 반환한다는 점**

In [8]:
lstm = LSTM(3, return_sequences=True, return_state=True)
hidden_states, last_hidden_state, last_cell_state = lstm(train_X)

print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('last hidden state : {}, shape: {}'.format(last_hidden_state, last_hidden_state.shape))
print('last cell state : {}, shape: {}'.format(last_cell_state, last_cell_state.shape))

hidden states : [[[ 0.00396451 -0.00716264 -0.00886327]
  [-0.02562587 -0.08779231 -0.0194691 ]
  [-0.0313523  -0.20313898 -0.02970505]
  [-0.06266455 -0.15132275 -0.04431514]]], shape: (1, 4, 3)
last hidden state : [[-0.06266455 -0.15132275 -0.04431514]], shape: (1, 3)
last cell state : [[-0.22602531 -0.54194105 -0.0599919 ]], shape: (1, 3)


* return_state =  True이므로 두 번째 출력값이 마지막 은닉 상태, 세 번째 출력값이 마지막 셀 상태인 것은 변함 없음
* 그런데 return_sequences = True이므로 첫번째 출력 값은 *모든 시점*의 은닉 상태 출력

### 4. Bidirectional(LSTM) 이해하기

In [9]:
k_init = tf.keras.initializers.Constant(value=0.1)
b_init = tf.keras.initializers.Constant(value=0)
r_init = tf.keras.initializers.Constant(value=0.1)

In [10]:
# return_sequences = False, return_state = True
bilstm = Bidirectional(LSTM(3, return_sequences=False, return_state=True, \
                            kernel_initializer=k_init, bias_initializer=b_init, recurrent_initializer=r_init))
hidden_states, forward_h, forward_c, backward_h, backward_c = bilstm(train_X)

print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('forward state : {}, shape: {}'.format(forward_h, forward_h.shape))
print('backward state : {}, shape: {}'.format(backward_h, backward_h.shape))

hidden states : [[0.63031393 0.63031393 0.63031393 0.7038734  0.7038734  0.7038734 ]], shape: (1, 6)
forward state : [[0.63031393 0.63031393 0.63031393]], shape: (1, 3)
backward state : [[0.7038734 0.7038734 0.7038734]], shape: (1, 3)


* 5개의 값을 반환
* return_state = True인 경우,
  * 정방향 LSTM의 은닉 상태와 셀 상태
  * 역방향 LSTM의 은닉 상태와 셀 상태 4가지를 반환
* 셀 상태는 forward_c와 backward_c에 저장하고 출력하지 않음  

* return_sequences = False인 경우, 정방향 LSTM의 마지막 시점인 은닉 상태와 역방향 LSTM의 첫번째 시점의 은닉 상태가 연결된 채 반환되기 때문  

* return_state = True인 경우, 반환한 은닉 상태의 값인 forward_h와 backward_h는 각각 정방향 LSTM의 마지막 시점인 은닉 상태와 역방향 LSTM의 첫번째 시점의 은닉 상태값. 이 두 값을 연결한 값이 hidden_states에 출력되는 값

In [11]:
# 은닉 상태의 값을 고정시켜두었기 때문에 return_sequences를 True로 할 경우, 출력이 어떻게 바뀌는지 비교가 가능

bilstm = Bidirectional(LSTM(3, return_sequences=True, return_state=True, \
                            kernel_initializer=k_init, bias_initializer=b_init, recurrent_initializer=r_init))
hidden_states, forward_h, forward_c, backward_h, backward_c = bilstm(train_X)

In [12]:
print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('forward state : {}, shape: {}'.format(forward_h, forward_h.shape))
print('backward state : {}, shape: {}'.format(backward_h, backward_h.shape))

hidden states : [[[0.35906473 0.35906473 0.35906473 0.7038734  0.7038734  0.7038734 ]
  [0.5511133  0.5511133  0.5511133  0.58863586 0.58863586 0.58863586]
  [0.59115744 0.59115744 0.59115744 0.3951699  0.3951699  0.3951699 ]
  [0.63031393 0.63031393 0.63031393 0.21942244 0.21942244 0.21942244]]], shape: (1, 4, 6)
forward state : [[0.63031393 0.63031393 0.63031393]], shape: (1, 3)
backward state : [[0.7038734 0.7038734 0.7038734]], shape: (1, 3)


* hidden states의 출력값에서는 이제 모든 시점의 은닉 상태가 출력됨
* 역방향 LSTM의 첫 번째 시점의 은닉 상태는 더 이상 정방향 LSTM의 마지막 시점의 은닉 상태와 연결되는 것이 아니라 정방향 LSTM의 첫번째 시점의 은닉 상태와 연결

### 5) RNN 언어 모델 (RNNLM)
* RNN으로 입력의 길이를 고정하지 않을 수 있음
* 훈련 기법: **교사 강요(teacher forcing)**
  * 훈련 과정에서는 A 훈련 샘플이 있다면, A 시퀀스를 모델의 입력으로 넣어 A를 예측하도록 훈련
  * 모델이 t 시점에서 예측한 값을 t+1 시점에 입력으로 넣지 않고, t 시점의 레이블, 즉 실제로 알고있는 '정답'을 t+1 시점의 입력으로 넣음
* 훈련 시 출력층에서 사용하는 활성화 함수: Softmax funciton
* 손실함수: Cross entropy function

### 6) Text Generation using RNN
* many-to-one 구조의 RNN으로 문맥을 반영하여 텍스트 생성
1. 데이터에 대한 이해와 전처리

In [13]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [14]:
text = """경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

In [15]:
# 단어 집합 생성하고 크기 확인
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
# 단어 집합의 크기를 저장할 때는 케라스 토크나이저의 정수 인코딩은 인덱스가 1부터 시작하지만, 패딩을 위한 0을 고려하여 +1
vocab_size = len(tokenizer.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 12


In [16]:
print(tokenizer.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [17]:
# 훈련 데이터 만들기
sequences = list()
for line in text.split('\n'): # Wn을 기준으로 문장 토큰화
    encoded = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수: %d' % len(sequences))

학습에 사용할 샘플의 개수: 11


In [18]:
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [20]:
# 전체 샘플에 대해서 길이를 일치
max_len = max(len(l) for l in sequences) # 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
print('샘플의 최대 길이 : {}'.format(max_len))
# 전체 샘플의 길이를 6으로 패딩
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences)

샘플의 최대 길이 : 6
[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [21]:
# 각 샘플의 마지막 단어를 레이블로 분리
sequences = np.array(sequences)
X = sequences[:,:-1] # 마지막 값을 제외하고 저장
y = sequences[:,-1] # 마지막 값만 저장

In [22]:
print(X)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]


In [23]:
print(y)

[ 3  1  4  5  1  7  1  9 10  1 11]


In [25]:
# 원-핫 인코딩
y = to_categorical(y, num_classes=vocab_size)
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


2. 모델 설계하기

In [26]:
# RNN 모델에 데이터 훈련
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

embedding_dim = 10
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
1/1 - 1s - loss: 2.5001 - accuracy: 0.0909
Epoch 2/200
1/1 - 0s - loss: 2.4835 - accuracy: 0.0909
Epoch 3/200
1/1 - 0s - loss: 2.4671 - accuracy: 0.0909
Epoch 4/200
1/1 - 0s - loss: 2.4508 - accuracy: 0.0909
Epoch 5/200
1/1 - 0s - loss: 2.4343 - accuracy: 0.0909
Epoch 6/200
1/1 - 0s - loss: 2.4177 - accuracy: 0.1818
Epoch 7/200
1/1 - 0s - loss: 2.4007 - accuracy: 0.1818
Epoch 8/200
1/1 - 0s - loss: 2.3832 - accuracy: 0.1818
Epoch 9/200
1/1 - 0s - loss: 2.3652 - accuracy: 0.2727
Epoch 10/200
1/1 - 0s - loss: 2.3466 - accuracy: 0.2727
Epoch 11/200
1/1 - 0s - loss: 2.3273 - accuracy: 0.4545
Epoch 12/200
1/1 - 0s - loss: 2.3073 - accuracy: 0.5455
Epoch 13/200
1/1 - 0s - loss: 2.2865 - accuracy: 0.5455
Epoch 14/200
1/1 - 0s - loss: 2.2649 - accuracy: 0.5455
Epoch 15/200
1/1 - 0s - loss: 2.2426 - accuracy: 0.5455
Epoch 16/200
1/1 - 0s - loss: 2.2195 - accuracy: 0.4545
Epoch 17/200
1/1 - 0s - loss: 2.1958 - accuracy: 0.4545
Epoch 18/200
1/1 - 0s - loss: 2.1715 - accuracy: 0.4545
E

In [27]:
# 모델이 정확하게 예측하고 있는지 문장을 생성하는 함수를 만들어서 출력
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        # 현재 단어에 대한 정수 인코딩과 패딩
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=5, padding='pre')
        # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items(): 
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면 break
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

In [28]:
# 입력된 단어로부터 다음 단어를 예측해서 문장을 생성하는 함수
print(sentence_generation(model, tokenizer, '경마장에', 4))

경마장에 있는 말이 뛰고 있다


In [29]:
print(sentence_generation(model, tokenizer, '그의', 2))

그의 말이 법이다


In [30]:
print(sentence_generation(model, tokenizer, '가는', 5))

가는 말이 고와야 오는 말이 곱다


* 앞의 문맥을 기준으로 '말이'라는 단어 다음에 나올 단어를 기존의 훈련 데이터와 일치하게 예측함을 보여줌

#### LSTM으로 텍스트 생성
1. 데이터에 대한 이해와 전처리

In [31]:
import pandas as pd
import numpy as np
from string import punctuation

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [38]:
from google.colab import files
uploaded = files.upload()

Saving ArticlesApril2017.csv to ArticlesApril2017.csv


In [39]:
df = pd.read_csv('/content/ArticlesApril2017.csv')
df.head()

,abstract,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,NaN,58def1347c459f24986d7c80,716,By STEPHEN HILTNER and SUSAN LEHMAN,article,Finding an Expansive View of a Forgotten Peop...,"['Photography', 'New York Times', 'Niger', 'Fe...",3,Insider,2,2017-04-01 00:15:41,Unknown,One of the largest photo displays in Times his...,The New York Times,News,https://www.nytimes.com/2017/03/31/insider/nig...
1,NaN,58def3237c459f24986d7c84,823,By GAIL COLLINS,article,"And Now, the Dreaded Trump Curse","['United States Politics and Government', 'Tru...",3,OpEd,23,2017-04-01 00:23:58,Unknown,Meet the gang from under the bus.,The New York Times,Op-Ed,https://www.nytimes.com/2017/03/31/opinion/and...
2,NaN,58def9f57c459f24986d7c90,575,By THE EDITORIAL BOARD,article,Venezuela’s Descent Into Dictatorship,"['Venezuela', 'Politics and Government', 'Madu...",3,Editorial,22,2017-04-01 00:53:06,Unknown,A court ruling annulling the legislature’s aut...,The New York Times,Editorial,https://www.nytimes.com/2017/03/31/opinion/ven...
3,NaN,58defd317c459f24986d7c95,1374,By MICHAEL POWELL,article,Stain Permeates Basketball Blue Blood,"['Basketball (College)', 'University of North ...",3,Sports,1,2017-04-01 01:06:52,College Basketball,"For two decades, until 2013, North Carolina en...",The New York Times,News,https://www.nytimes.com/2017/03/31/sports/ncaa...
4,NaN,58df09b77c459f24986d7ca7,708,By DEB AMLEN,article,Taking Things for Granted,['Crossword Puzzles'],3,Games,0,2017-04-01 02:00:14,Unknown,In which Howard Barkin and Will Shortz teach u...,The New York Times,News,https://www.nytimes.com/2017/03/31/crosswords/...


In [40]:
print('열의 개수: ',len(df.columns))
print(df.columns)

열의 개수:  16
Index(['abstract', 'articleID', 'articleWordCount', 'byline', 'documentType',
       'headline', 'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [41]:
# Null값 있는지 확인
print(df['headline'].isnull().values.any())

False


In [42]:
headline = []
# 헤드라인의 값들을 리스트로 저장
headline.extend(list(df.headline.values)) 
headline[:5]

['Finding an Expansive View  of a Forgotten People in Niger',
 'And Now,  the Dreaded Trump Curse',
 'Venezuela’s Descent Into Dictatorship',
 'Stain Permeates Basketball Blue Blood',
 'Taking Things for Granted']

In [43]:
# 노이즈 제거
print('총 샘플의 개수 : {}'.format(len(headline)))

headline = [word for word in headline if word != "Unknown"]
print('노이즈값 제거 후 샘플의 개수 : {}'.format(len(headline)))

총 샘플의 개수 : 886
노이즈값 제거 후 샘플의 개수 : 831


In [44]:
headline[:5]

['Finding an Expansive View  of a Forgotten People in Niger',
 'And Now,  the Dreaded Trump Curse',
 'Venezuela’s Descent Into Dictatorship',
 'Stain Permeates Basketball Blue Blood',
 'Taking Things for Granted']

In [45]:
# 데이터 전처리
def repreprocessing(raw_sentence):
    preproceseed_sentence = raw_sentence.encode("utf8").decode("ascii",'ignore')
    # 구두점 제거와 동시에 소문자화
    return ''.join(word for word in preproceseed_sentence if word not in punctuation).lower()

preporcessed_headline = [repreprocessing(x) for x in headline]
preporcessed_headline[:5]

['finding an expansive view  of a forgotten people in niger',
 'and now  the dreaded trump curse',
 'venezuelas descent into dictatorship',
 'stain permeates basketball blue blood',
 'taking things for granted']

In [46]:
# 단어 집합 만들고 크기 확인
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preporcessed_headline)
vocab_size = len(tokenizer.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 2422


In [47]:
# 정수 인코딩과 하나의 문장을 분해하여 훈련 데이터 구성
sequences = list()

for sentence in preporcessed_headline:

    # 각 샘플에 대한 정수 인코딩
    encoded = tokenizer.texts_to_sequences([sentence])[0] 
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11]

[[169, 17],
 [169, 17, 665],
 [169, 17, 665, 367],
 [169, 17, 665, 367, 4],
 [169, 17, 665, 367, 4, 2],
 [169, 17, 665, 367, 4, 2, 666],
 [169, 17, 665, 367, 4, 2, 666, 170],
 [169, 17, 665, 367, 4, 2, 666, 170, 5],
 [169, 17, 665, 367, 4, 2, 666, 170, 5, 667],
 [6, 80],
 [6, 80, 1]]

In [48]:
# 어떤 정수가 어떤 단어를 의미하는지 알아보기 위해 인덱스로부터 단어를 찾는 index_to_word 만들기
index_to_word = {}
for key, value in tokenizer.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print('빈도수 상위 582번 단어 : {}'.format(index_to_word[582]))

빈도수 상위 582번 단어 : exist


In [49]:
# 샘플 길이 패딩
max_len = max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

샘플의 최대 길이 : 19
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 169
   17]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 169  17
  665]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 169  17 665
  367]]


In [50]:
# 맨 우측 단어로만 레이블로 분리
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

print(X[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 169]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 169  17]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 169  17 665]]


In [51]:
print(y[:3])

[ 17 665 367]


In [52]:
# 레이블 데이터 y에 대해서 원-핫 인코딩 수ㅐㅇ
y = to_categorical(y, num_classes=vocab_size)

2. 모델 설계하기

In [53]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [54]:
embedding_dim = 10
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
151/151 - 5s - loss: 7.4316 - accuracy: 0.0279
Epoch 2/200
151/151 - 4s - loss: 6.9209 - accuracy: 0.0323
Epoch 3/200
151/151 - 4s - loss: 6.7968 - accuracy: 0.0318
Epoch 4/200
151/151 - 4s - loss: 6.7105 - accuracy: 0.0343
Epoch 5/200
151/151 - 3s - loss: 6.6251 - accuracy: 0.0422
Epoch 6/200
151/151 - 3s - loss: 6.5215 - accuracy: 0.0470
Epoch 7/200
151/151 - 3s - loss: 6.3951 - accuracy: 0.0497
Epoch 8/200
151/151 - 4s - loss: 6.2533 - accuracy: 0.0497
Epoch 9/200
151/151 - 4s - loss: 6.1000 - accuracy: 0.0516
Epoch 10/200
151/151 - 4s - loss: 5.9394 - accuracy: 0.0556
Epoch 11/200
151/151 - 3s - loss: 5.7809 - accuracy: 0.0583
Epoch 12/200
151/151 - 3s - loss: 5.6288 - accuracy: 0.0676
Epoch 13/200
151/151 - 3s - loss: 5.4870 - accuracy: 0.0680
Epoch 14/200
151/151 - 4s - loss: 5.3530 - accuracy: 0.0699
Epoch 15/200
151/151 - 3s - loss: 5.2256 - accuracy: 0.0747
Epoch 16/200
151/151 - 3s - loss: 5.1029 - accuracy: 0.0789
Epoch 17/200
151/151 - 3s - loss: 4.9818 - accura

In [61]:
# 문장 생성
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre')

        # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items(): 
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

In [62]:
print(sentence_generation(model, tokenizer, 'i', 10))

i love my sublet and i want to buy it how


In [63]:
print(sentence_generation(model, tokenizer, 'how', 10))

how to be mindful while gardening and a next thinks bill
